In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("lib")

import lib

import torch
torch.set_default_dtype(torch.float64)

from functorch import vmap

In [2]:
file = pd.read_csv('burger_2d_500.csv')

In [3]:
vel_0 = file['vel_0'].to_numpy()
vel_1 = file['vel_1'].to_numpy()

In [4]:
data = np.concatenate((vel_0, vel_1))
data = data.reshape((len(data), 1))
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = data.T

u_train = np.copy(data)
u_train = torch.tensor(u_train)

In [5]:
N = u_train.shape[1]
n = 20

In [6]:
u_train.shape

torch.Size([16, 29282])

In [7]:
nrbs = lib.nrbs.NRBS(N=N, n=n, mu=N/40)

In [8]:
encoded = nrbs.encode(u_train)
decoded = nrbs.decode(encoded)

bubbles shape:  torch.Size([20, 29282, 1465])


In [9]:
encoded.shape

torch.Size([16, 20])

In [10]:
decoded.shape

torch.Size([16, 29282])

In [11]:
nrbs.bandwidth.shape

torch.Size([20, 29282])

In [12]:
vmap_bubble = vmap(nrbs.bubble, in_dims=0)
vmap_vmap_bubble = vmap(vmap_bubble, in_dims=0)

In [13]:
test = vmap_bubble(nrbs.bandwidth[0, :])
test_test = vmap_vmap_bubble(nrbs.bandwidth)

In [14]:
test.shape

torch.Size([29282, 1465])

In [15]:
test_test.shape

torch.Size([20, 29282, 1465])